In [ ]:
import helpers
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.optim import lr_scheduler
import pandas as pd

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Change paths
train_path = 'train_set_augmented.parquet'
test_path = 'test_set.parquet'

train_df = pd.read_parquet(train_path, engine='pyarrow')
test_df = pd.read_parquet(test_path, engine='pyarrow')

In [ ]:
BLOCK_LENGTH = 1
test_video_indices = test_df.groupby('video_id').size().values
test_video_indices = np.insert(test_video_indices, 0, 0)
test_video_indices = np.cumsum(test_video_indices)

train_video_indices = train_df.groupby('video_id').size().values
train_video_indices = np.insert(train_video_indices, 0, 0)
train_video_indices = np.cumsum(train_video_indices)

set_mapping = {
    'Correct' : 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6
}

In [ ]:
X_train, X_test, y_train, y_test = helpers.import_data_set(train_df, test_df, set_mapping, True)

In [ ]:
X_train = helpers.pad_sequences(X_train, BLOCK_LENGTH, train_video_indices)
X_train = X_train.reshape(-1,1,BLOCK_LENGTH,107)
X_test = helpers.pad_sequences(X_test, BLOCK_LENGTH, test_video_indices)
X_test = X_test.reshape(-1,1,BLOCK_LENGTH,107)
y_train = helpers.pad_sequences(y_train, BLOCK_LENGTH, train_video_indices)
y_train = y_train[::BLOCK_LENGTH].squeeze()
y_test = helpers.pad_sequences(y_test, BLOCK_LENGTH, test_video_indices)
y_test = y_test[::BLOCK_LENGTH].squeeze()

In [ ]:
video_indices = test_df.groupby('video_id').size().values
video_indices = np.ceil(video_indices / BLOCK_LENGTH)
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices).astype(int)

We define the CNN models as follows

In [ ]:
get_cnn2b = lambda: torch.nn.Sequential(
    # Block 1
    nn.Conv2d(1, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Block 2
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),

    torch.nn.Flatten(),
    torch.nn.Linear(832 * BLOCK_LENGTH, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7)
    )

In [ ]:
get_cnn3b = lambda: torch.nn.Sequential(
    # Block 1
    nn.Conv2d(1, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Block 2
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Block 3
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Fully connected layers
    nn.Flatten(),
    nn.Linear(53248, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 256),  # Adjust the output size based on your task
    nn.ReLU(inplace=True),
    nn.Linear(256, 7),
    nn.LogSoftmax(dim=1)
)

In [ ]:
# Define the model, change this if you want to test cnn3b
model = get_cnn2b()

In [ ]:
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Running on {device}")
# Create an instance of the custom dataset
dataset = TensorDataset(X_train, y_train)

# Create a DataLoader
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model_path = ''
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


In [ ]:
for i in range(50):
  helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path, device)
  helpers.test_accuracy(model, X_test,y_test,video_indices, device)

Processing: 64.25%

In [ ]:
helpers.test_accuracy(model, X_test,y_test,video_indices, device)